In [1]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix, plot_stat_map
import os
import pandas as pd
import pickle
import re

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/01_level1')
from level1_utils import run_level1, make_level1_design_matrix, get_events

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/'
behavior_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/behavioral_data/all_trials.csv'

subnum = '01'
runnum='4'

In [7]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 0)
# plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,fractalProb_ev,reward_ev,stim_ev
choice_st,1.000000,-0.796205,0.883857,0.415893,0.839958
cross_ev,-0.796205,1.000000,-0.549370,-0.711303,-0.789923
fractalProb_ev,0.883857,-0.549370,1.000000,0.027088,0.645227
reward_ev,0.415893,-0.711303,0.027088,1.000000,0.498400
stim_ev,0.839958,-0.789923,0.645227,0.498400,1.000000


In [8]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,cross_ev,fractalProb_ev,reward_ev,stim_ev
choiceShift_st,1.000000,-0.887208,0.590639,0.757393,0.837817
cross_ev,-0.887208,1.000000,-0.549370,-0.711303,-0.789923
fractalProb_ev,0.590639,-0.549370,1.000000,0.027088,0.645227
reward_ev,0.757393,-0.711303,0.027088,1.000000,0.498400
stim_ev,0.837817,-0.789923,0.645227,0.498400,1.000000


In [5]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,cross_rt,fractalProb_ev,reward_ev,stim_ev,stim_rt
choice_st,1.000000,-0.723174,-0.205398,0.883857,0.415893,0.973426,-0.000522
cross_ev,-0.723174,1.000000,0.000112,-0.507547,-0.634884,-0.780758,-0.032182
cross_rt,-0.205398,0.000112,1.000000,-0.248806,-0.101781,-0.181907,-0.088604
fractalProb_ev,0.883857,-0.507547,-0.248806,1.000000,0.027088,0.761268,0.001233
reward_ev,0.415893,-0.634884,-0.101781,0.027088,1.000000,0.590911,-0.096463
stim_ev,0.973426,-0.780758,-0.181907,0.761268,0.590911,1.000000,0.000264
stim_rt,-0.000522,-0.032182,-0.088604,0.001233,-0.096463,0.000264,1.000000


In [6]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1a', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,cross_ev,cross_rt,fractalProb_ev,reward_ev,stim_ev,stim_rt
choiceShift_st,1.000000,-0.807940,-0.168860,0.590639,0.757393,0.955171,-0.033936
cross_ev,-0.807940,1.000000,0.000112,-0.507547,-0.634884,-0.780758,-0.032182
cross_rt,-0.168860,0.000112,1.000000,-0.248806,-0.101781,-0.181907,-0.088604
fractalProb_ev,0.590639,-0.507547,-0.248806,1.000000,0.027088,0.761268,0.001233
reward_ev,0.757393,-0.634884,-0.101781,0.027088,1.000000,0.590911,-0.096463
stim_ev,0.955171,-0.780758,-0.181907,0.761268,0.590911,1.000000,0.000264
stim_rt,-0.033936,-0.032182,-0.088604,0.001233,-0.096463,0.000264,1.000000


In [ ]:
design_matrix = make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
mnum = 'model1'
reg_rt = 1
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt)

# run_level1(subnum, mnum, data_path, behavior_path, out_path, regress_rt = reg_rt)

In [ ]:
mnum = 'model2'
reg_rt = 0
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt))
                                                                                
# run_level1(subnum, 'model2', data_path, behavior_path, out_path, regress_rt = 0)

In [29]:
mnum = 'model2'
reg_rt = 0
sub_paths = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s/*")%(mnum, str(reg_rt))
len(glob.glob(os.path.join(sub_paths, '*')))

11